In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
import statsmodels.api as sm

font_path = 'C:\\Windows\\Fonts\\gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
mpl.rc('font', family=font)

In [185]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pandas as pd

def divide_time_range(time):
    if time < 600:
        return '00:00~06:00'
    elif time < 1000:
        return '06:00~10:00' # 출근시간
    elif time < 1700:
        return '10:00~17:00'
    elif time < 2100:
        return '17:00~21:00' # 퇴근시간
    else:
        return '21:00~24:00'

def check_best_sarima(data,p,d,q,P,D,Q):
    #plot_acf(data)
    #plot_pacf(data)
    #plt.show()

    model = SARIMAX(data.values, order=(p, d, q), seasonal_order=(P, D, Q, 5))  # 데이터를 넘파이 배열로 변환하여 사용
    model_fit = model.fit()

    forecast = model_fit.forecast(steps=1)[0]
    
    # 예측 결과 출력
    #print(model_fit.summary())

    return forecast

def sarima_test(data, IDs,p,d,q,P,D,Q,a,b,c):
    result = []
    for 주거_ID in IDs:
        주거_data = data[data['대여소_ID'] == 주거_ID]
        주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
        주거_data.set_index('기준_날짜', inplace=True)
        arima_data = 주거_data.resample('D')['전체_건수'].sum()
        
        forecast = check_best_sarima(arima_data,p,d,q,P,D,Q)
        result.append([주거_ID, forecast,a,b,c])
        
    return result

# 주거_강서_df1 : 대여 ARIMA(4,0,0)(2,1,1)[5] x                주거-대여-출근 
# 주거_강서_df1 : 반납 ARIMA(0,0,0)(2,1,0)[5] x                주거-반납-출근 
# 주거_강서_df2 : 대여 ARIMA(2,0,3)(2,1,0)[5] x                주거-대여-퇴근 
# 주거_강서_df2 : 반납 ARIMA(0,0,1)(2,1,1)[5] x                주거-반납-퇴근 
# 상업_강서_df1 : 대여 ARIMA(4,0,2)(2,1,0)[5] x                상업-대여-출근 ==오류가 생겼음
# 상업_강서_df1 : 반납 ARIMA(3,0,3)(1,1,0)[5] x                상업-반납-출근 
# 상업_강서_df2 : 대여 ARIMA(4,0,2)(2,1,0)[5] x                상업-대여-퇴근 
# 상업_강서_df2 : 반납 ARIMA(4,0,2)(2,1,0)[5] x                상업-반납-퇴근 

# 주거 + 대여 + 출근
# 주거_강서_df1 : 대여 ARIMA(4,0,0)(2,1,1)[5]

In [168]:
강서_대여소_ID = pd.read_csv('../../Data/따릉이/강서_따릉이_대여소_ID.csv',encoding='utf-8')

강서_데이터1 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_01.csv',encoding='utf-8')
강서_데이터2 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_02.csv',encoding='utf-8')

강서_데이터 = pd.concat([강서_데이터1, 강서_데이터2], ignore_index=True)
print(len(강서_데이터))

top_20_ID = pd.read_csv('../../Data/따릉이/쏠림정도_top20_대여소.csv',encoding='utf-8')
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(주거_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '06:00~10:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['시작_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'시작_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납
check_forcast =[]
check_forcast.append(sarima_test(강서_df1,주거_IDs,2,1,3,2,1,0,"주거",'06:00~10:00','대여'))


3385613


C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

# 주거 반납 출근
# 주거_강서_df1 : 반납 ARIMA(0,0,0)(2,1,0)[5] 

In [169]:

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(주거_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '06:00~10:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

check_forcast.append(sarima_test(강서_df1,주거_IDs,0,0,0,2,1,0,"주거",'06:00~10:00','반납'))
print(check_forcast)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

[[['ST-1253', 41.75640128865487, '주거', '06:00~10:00', '대여'], ['ST-1352', 47.56770326672436, '주거', '06:00~10:00', '대여'], ['ST-1645', 34.6482530284084, '주거', '06:00~10:00', '대여'], ['ST-2050', 44.358865189269935, '주거', '06:00~10:00', '대여'], ['ST-2056', 39.47565616510094, '주거', '06:00~10:00', '대여'], ['ST-2557', 53.08812893766529, '주거', '06:00~10:00', '대여'], ['ST-2965', 38.047462325016106, '주거', '06:00~10:00', '대여']], [['ST-1253', 5.510320896646613, '주거', '06:00~10:00', '반납'], ['ST-1352', 14.719196342846706, '주거', '06:00~10:00', '반납'], ['ST-1645', 9.912933717026796, '주거', '06:00~10:00', '반납'], ['ST-2050', 6.8953633904563665, '주거', '06:00~10:00', '반납'], ['ST-2056', 16.892619627281896, '주거', '06:00~10:00', '반납'], ['ST-2557', 36.15943095229401, '주거', '06:00~10:00', '반납'], ['ST-2965', 12.449725602188366, '주거', '06:00~10:00', '반납']]]


C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])


# 주거 + 대여 + 퇴근
# 주거_강서_df2 : 대여 ARIMA(2,0,3)(2,1,0)[5]   

In [170]:

강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(주거_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '17:00~21:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['시작_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'시작_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

check_forcast.append(sarima_test(강서_df1,주거_IDs,2,0,3,2,1,0,"주거",'17:00~21:00','대여'))
print(check_forcast)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

[[['ST-1253', 41.75640128865487, '주거', '06:00~10:00', '대여'], ['ST-1352', 47.56770326672436, '주거', '06:00~10:00', '대여'], ['ST-1645', 34.6482530284084, '주거', '06:00~10:00', '대여'], ['ST-2050', 44.358865189269935, '주거', '06:00~10:00', '대여'], ['ST-2056', 39.47565616510094, '주거', '06:00~10:00', '대여'], ['ST-2557', 53.08812893766529, '주거', '06:00~10:00', '대여'], ['ST-2965', 38.047462325016106, '주거', '06:00~10:00', '대여']], [['ST-1253', 5.510320896646613, '주거', '06:00~10:00', '반납'], ['ST-1352', 14.719196342846706, '주거', '06:00~10:00', '반납'], ['ST-1645', 9.912933717026796, '주거', '06:00~10:00', '반납'], ['ST-2050', 6.8953633904563665, '주거', '06:00~10:00', '반납'], ['ST-2056', 16.892619627281896, '주거', '06:00~10:00', '반납'], ['ST-2557', 36.15943095229401, '주거', '06:00~10:00', '반납'], ['ST-2965', 12.449725602188366, '주거', '06:00~10:00', '반납']], [['ST-1253', 2.197098294886386, '주거', '17:00~21:00', '대여'], ['ST-1352', 17.465192215548022, '주거', '17:00~21:00', '대여'], ['ST-1645', 2.834577840768213, '주거', '17:00~

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


# 주거 + 반납 + 퇴근
# 주거_강서_df2 : 반납 ARIMA(0,0,1)(2,1,1)[5]

In [171]:

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(주거_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '17:00~21:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

check_forcast.append(sarima_test(강서_df1,주거_IDs,2,0,3,2,1,0,"주거",'17:00~21:00','반납'))
print(check_forcast)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

[[['ST-1253', 41.75640128865487, '주거', '06:00~10:00', '대여'], ['ST-1352', 47.56770326672436, '주거', '06:00~10:00', '대여'], ['ST-1645', 34.6482530284084, '주거', '06:00~10:00', '대여'], ['ST-2050', 44.358865189269935, '주거', '06:00~10:00', '대여'], ['ST-2056', 39.47565616510094, '주거', '06:00~10:00', '대여'], ['ST-2557', 53.08812893766529, '주거', '06:00~10:00', '대여'], ['ST-2965', 38.047462325016106, '주거', '06:00~10:00', '대여']], [['ST-1253', 5.510320896646613, '주거', '06:00~10:00', '반납'], ['ST-1352', 14.719196342846706, '주거', '06:00~10:00', '반납'], ['ST-1645', 9.912933717026796, '주거', '06:00~10:00', '반납'], ['ST-2050', 6.8953633904563665, '주거', '06:00~10:00', '반납'], ['ST-2056', 16.892619627281896, '주거', '06:00~10:00', '반납'], ['ST-2557', 36.15943095229401, '주거', '06:00~10:00', '반납'], ['ST-2965', 12.449725602188366, '주거', '06:00~10:00', '반납']], [['ST-1253', 2.197098294886386, '주거', '17:00~21:00', '대여'], ['ST-1352', 17.465192215548022, '주거', '17:00~21:00', '대여'], ['ST-1645', 2.834577840768213, '주거', '17:00~

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


# 상업 + 대여 + 출근
# 상업_강서_df1 : 대여 ARIMA(4,0,2)(2,1,0)[5]
 - 오류생김 3,0,2, 2,1,0, 대체


In [193]:

강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(상업_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '06:00~10:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['시작_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'시작_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납
check_forcast1=[]
check_forcast1.append(sarima_test(강서_df1,상업_IDs,3,0,2,2,1,0,"상업",'06:00~10:00','대여'))
print(check_forcast)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126

LinAlgError: LU decomposition error.

# 상업 + 반납 + 출근
# 상업_강서_df1 : 반납 ARIMA(3,0,3)(1,1,0)[5] 


In [173]:

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(상업_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '06:00~10:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

check_forcast.append(sarima_test(강서_df1,상업_IDs,3,0,3,1,1,0,"상업",'06:00~10:00','반납'))
print(check_forcast)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

[[['ST-1253', 41.75640128865487, '주거', '06:00~10:00', '대여'], ['ST-1352', 47.56770326672436, '주거', '06:00~10:00', '대여'], ['ST-1645', 34.6482530284084, '주거', '06:00~10:00', '대여'], ['ST-2050', 44.358865189269935, '주거', '06:00~10:00', '대여'], ['ST-2056', 39.47565616510094, '주거', '06:00~10:00', '대여'], ['ST-2557', 53.08812893766529, '주거', '06:00~10:00', '대여'], ['ST-2965', 38.047462325016106, '주거', '06:00~10:00', '대여']], [['ST-1253', 5.510320896646613, '주거', '06:00~10:00', '반납'], ['ST-1352', 14.719196342846706, '주거', '06:00~10:00', '반납'], ['ST-1645', 9.912933717026796, '주거', '06:00~10:00', '반납'], ['ST-2050', 6.8953633904563665, '주거', '06:00~10:00', '반납'], ['ST-2056', 16.892619627281896, '주거', '06:00~10:00', '반납'], ['ST-2557', 36.15943095229401, '주거', '06:00~10:00', '반납'], ['ST-2965', 12.449725602188366, '주거', '06:00~10:00', '반납']], [['ST-1253', 2.197098294886386, '주거', '17:00~21:00', '대여'], ['ST-1352', 17.465192215548022, '주거', '17:00~21:00', '대여'], ['ST-1645', 2.834577840768213, '주거', '17:00~

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


# 상업 + 대여 + 퇴근
# 상업_강서_df2 : 대여 ARIMA(4,0,2)(2,1,0)[5]

In [174]:

강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(상업_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '17:00~21:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['시작_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'시작_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

check_forcast.append(sarima_test(강서_df1,상업_IDs,4,0,2,2,1,0,"상업",'17:00~21:00','대여'))
print(check_forcast)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126

[[['ST-1253', 41.75640128865487, '주거', '06:00~10:00', '대여'], ['ST-1352', 47.56770326672436, '주거', '06:00~10:00', '대여'], ['ST-1645', 34.6482530284084, '주거', '06:00~10:00', '대여'], ['ST-2050', 44.358865189269935, '주거', '06:00~10:00', '대여'], ['ST-2056', 39.47565616510094, '주거', '06:00~10:00', '대여'], ['ST-2557', 53.08812893766529, '주거', '06:00~10:00', '대여'], ['ST-2965', 38.047462325016106, '주거', '06:00~10:00', '대여']], [['ST-1253', 5.510320896646613, '주거', '06:00~10:00', '반납'], ['ST-1352', 14.719196342846706, '주거', '06:00~10:00', '반납'], ['ST-1645', 9.912933717026796, '주거', '06:00~10:00', '반납'], ['ST-2050', 6.8953633904563665, '주거', '06:00~10:00', '반납'], ['ST-2056', 16.892619627281896, '주거', '06:00~10:00', '반납'], ['ST-2557', 36.15943095229401, '주거', '06:00~10:00', '반납'], ['ST-2965', 12.449725602188366, '주거', '06:00~10:00', '반납']], [['ST-1253', 2.197098294886386, '주거', '17:00~21:00', '대여'], ['ST-1352', 17.465192215548022, '주거', '17:00~21:00', '대여'], ['ST-1645', 2.834577840768213, '주거', '17:00~

# 상업 + 반납 + 퇴근
# 상업_강서_df2 : 반납 ARIMA(4,0,2)(2,1,0)[5]

In [175]:

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(상업_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '17:00~21:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

check_forcast.append(sarima_test(강서_df1,상업_IDs,4,0,2,2,1,0,"상업",'17:00~21:00','반납'))
print(check_forcast)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126933364.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ciw96\AppData\Local\Temp\ipykernel_54832\1126

[[['ST-1253', 41.75640128865487, '주거', '06:00~10:00', '대여'], ['ST-1352', 47.56770326672436, '주거', '06:00~10:00', '대여'], ['ST-1645', 34.6482530284084, '주거', '06:00~10:00', '대여'], ['ST-2050', 44.358865189269935, '주거', '06:00~10:00', '대여'], ['ST-2056', 39.47565616510094, '주거', '06:00~10:00', '대여'], ['ST-2557', 53.08812893766529, '주거', '06:00~10:00', '대여'], ['ST-2965', 38.047462325016106, '주거', '06:00~10:00', '대여']], [['ST-1253', 5.510320896646613, '주거', '06:00~10:00', '반납'], ['ST-1352', 14.719196342846706, '주거', '06:00~10:00', '반납'], ['ST-1645', 9.912933717026796, '주거', '06:00~10:00', '반납'], ['ST-2050', 6.8953633904563665, '주거', '06:00~10:00', '반납'], ['ST-2056', 16.892619627281896, '주거', '06:00~10:00', '반납'], ['ST-2557', 36.15943095229401, '주거', '06:00~10:00', '반납'], ['ST-2965', 12.449725602188366, '주거', '06:00~10:00', '반납']], [['ST-1253', 2.197098294886386, '주거', '17:00~21:00', '대여'], ['ST-1352', 17.465192215548022, '주거', '17:00~21:00', '대여'], ['ST-1645', 2.834577840768213, '주거', '17:00~

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [186]:
check1 = pd.read_csv('tpss_bcycl_od_statnhm_20240401.csv',encoding='cp949')

check1 = check1[check1['집계_기준'] != '도착시간']
check1 = check1[(check1['전체_이용_분'] != 0) & (check1['전체_이용_거리'] != 0)]
    
check1['시간대_구간'] = check1['기준_시간대'].apply(divide_time_range)

check = []
check2 = check1[check1['시간대_구간'] == '06:00~10:00']

for 주거_ID in 주거_IDs:
    check.append(check2[check2['시작_대여소_ID']==주거_ID]['전체_건수'].sum())

for 주거_ID in 주거_IDs:
    check.append(check2[check2['종료_대여소_ID']==주거_ID]['전체_건수'].sum())
 
check2 = check1[check1['시간대_구간'] == '17:00~21:00']

for 주거_ID in 주거_IDs:
    check.append(check2[check2['시작_대여소_ID']==주거_ID]['전체_건수'].sum())

for 주거_ID in 주거_IDs:
    check.append(check2[check2['종료_대여소_ID']==주거_ID]['전체_건수'].sum())
 
check2 = check1[check1['시간대_구간'] == '06:00~10:00']

for 주거_ID in 상업_IDs:
    check.append(check2[check2['시작_대여소_ID']==주거_ID]['전체_건수'].sum())

for 주거_ID in 상업_IDs:
    check.append(check2[check2['종료_대여소_ID']==주거_ID]['전체_건수'].sum())
 
check2 = check1[check1['시간대_구간'] == '17:00~21:00']

for 주거_ID in 상업_IDs:
    check.append(check2[check2['시작_대여소_ID']==주거_ID]['전체_건수'].sum())
    
for 주거_ID in 상업_IDs:
    check.append(check2[check2['종료_대여소_ID']==주거_ID]['전체_건수'].sum())
 

In [192]:
# DataFrame으로 변환
df_list = []
for sublist in check_forcast:
    for item in sublist:
        df_list.append(item)

df = pd.DataFrame(df_list, columns=['대여소_ID', '예측값', '지구', '시간대_구간', '출퇴'])
df['실제 값'] = check

df[df['지구']=='주거']

,대여소_ID,예측값,지구,시간대_구간,출퇴,실제 값
0,ST-1253,41.756401,주거,06:00~10:00,대여,32
1,ST-1352,47.567703,주거,06:00~10:00,대여,56
2,ST-1645,34.648253,주거,06:00~10:00,대여,54
3,ST-2050,44.358865,주거,06:00~10:00,대여,52
4,ST-2056,39.475656,주거,06:00~10:00,대여,34
5,ST-2557,53.088129,주거,06:00~10:00,대여,53
6,ST-2965,38.047462,주거,06:00~10:00,대여,54
7,ST-1253,5.510321,주거,06:00~10:00,반납,3
8,ST-1352,14.719196,주거,06:00~10:00,반납,13
9,ST-1645,9.912934,주거,06:00~10:00,반납,8


[    대여소_ID        예측값  지구 대여_반납  출퇴
 0  ST-1253  41.756401  주거    대여  출근
 1  ST-1352  47.567703  주거    대여  출근
 2  ST-1645  34.648253  주거    대여  출근
 3  ST-2050  44.358865  주거    대여  출근
 4  ST-2056  39.475656  주거    대여  출근
 5  ST-2557  53.088129  주거    대여  출근
 6  ST-2965  38.047462  주거    대여  출근]

In [187]:
check

[32,
 56,
 54,
 52,
 34,
 53,
 54,
 3,
 13,
 8,
 13,
 21,
 31,
 13,
 33,
 34,
 32,
 22,
 36,
 63,
 36,
 63,
 60,
 68,
 76,
 61,
 87,
 51,
 33,
 10,
 4,
 151,
 6,
 9,
 7,
 7,
 75,
 12,
 26,
 3,
 19,
 64,
 54,
 76,
 174,
 147,
 101,
 83,
 37,
 76,
 48,
 72,
 80,
 161,
 63,
 49,
 55,
 226,
 79,
 89,
 40,
 33,
 61,
 47,
 49,
 66,
 82,
 43,
 16,
 9,
 169,
 20,
 33,
 8,
 17,
 53,
 22,
 51,
 5,
 8]